## db engine

In [2]:
from flask_sqlalchemy import SQLAlchemy
from flask import Flask
from datetime import datetime
import os
import re

basedir = os.path.abspath('..')
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'postgresql+psycopg2://localhost/test_db'
app.config['SQLALCHEMY_COMMIT_ON_TEARDOWN'] = True
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['TESTING_FOLDER'] = os.path.join(basedir, 'utility/testing')
app.config['MYDICT_FOLDER'] = os.path.join(basedir, 'utility/mydict')
db = SQLAlchemy(app)

### db model

In [3]:
class Article(db.Model):
    __tablename__ = 'articles'

    id = db.Column(db.Integer, primary_key=True)
    article = db.Column(db.String(256), unique=True)

    def __init__(self, **kwargs):
        super(Article, self).__init__(**kwargs)

    def __repr__(self):
        return f'<Article:{self.id}-{self.article}>'

class Sentence(db.Model):
    __tablename__ = 'sentences'
    id = db.Column(db.Integer, primary_key=True)
    sentence = db.Column(db.String(256))
    translation = db.Column(db.String(256))
    article_id = db.Column(db.Integer, db.ForeignKey('articles.id'))
    article = db.relationship('Article', backref='sentences')

    def __init__(self, **kwargs):
        super(Sentence, self).__init__(**kwargs)

    def __repr__(self):
        return f'<Sentence:{self.id}-{self.sentence}>'

class Word(db.Model):
    __tablename__ = 'words'
    id = db.Column(db.Integer, primary_key=True)
    word = db.Column(db.String(64), unique=True)
    translation = db.Column(db.String(256))

    def __init__(self, **kwargs):
        super(Word, self).__init__(**kwargs)

    def __repr__(self):
        return f'<Word:{self.id}-{self.word}>'

class SentenceWord(db.Model):
    __tablename__ = 'sentencewords'
    id = db.Column(db.Integer, primary_key=True)
    sentence_id = db.Column(db.Integer, db.ForeignKey('sentences.id'))
    word_id = db.Column(db.Integer, db.ForeignKey('words.id'))
    sentence = db.relationship('Sentence', backref='sentencewords')
    word = db.relationship('Word', backref='sentencewords')

    def __init__(self, **kwargs):
        super(SentenceWord, self).__init__(**kwargs)

    def __repr__(self):
        return f'<SentenceWord:{self.id}-{self.sentence}-{self.word}>'

class Review(db.Model):
    __tablename__ = 'reviews'
    id = db.Column(db.Integer, primary_key=True)
    word_id = db.Column(db.Integer, db.ForeignKey('words.id'))
    noshow = db.Column(db.Boolean, default=False)
    known = db.Column(db.Boolean, default=False)
    unknown = db.Column(db.Boolean, default=True)
    blurry = db.Column(db.Boolean, default=False)
    review_timestamp = db.Column(db.DateTime, default=datetime.utcnow,onupdate=datetime.now())
    word = db.relationship('Word', backref='review')

    def __init__(self, **kwargs):
        super(Review, self).__init__(**kwargs)

    def __repr__(self):
        return f'<Review:{self.id}-{self.word}>'

class Mydict(db.Model):
    __tablename__ = 'mydict'
    id = db.Column(db.Integer, primary_key=True)
    word_id = db.Column(db.Integer, db.ForeignKey('words.id'))
    word = db.relationship('Word', backref='mydict')
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def __init__(self, **kwargs):
        super(Mydict, self).__init__(**kwargs)

    def __repr__(self):
        return f'<Mydict:{self.id}-{self.word}>'

### define functions

In [7]:
def get_sentence(file):
    with open(file, 'r') as f:
        text = f.read()
    pttn = re.compile(r"[a-zA-Z].*", re.I)
    sentences = re.findall(pttn, text)
    return sentences

def get_tokens(text):
    tokens = re.findall('[a-z]+', text.lower())
    token = list(dict.fromkeys(tokens))
    return token

def read_text(filename):
    with open(filename, 'r') as f:
        text = f.read()
    return text

def delete_table_records():
    db.session.query(SentenceWord).delete()
    db.session.query(Sentence).delete()
    db.session.query(Word).delete()
    db.session.query(Article).delete()
    db.session.commit()

def drop_everything():
    """(On a live db) drops all foreign key constraints before dropping all tables.
    Workaround for SQLAlchemy not doing DROP ## CASCADE for drop_all()
    (https://github.com/pallets/flask-sqlalchemy/issues/722)
    """
    from sqlalchemy.engine.reflection import Inspector
    from sqlalchemy.schema import DropConstraint, DropTable, MetaData, Table
    con = db.engine.connect()
    print(con)
    trans = con.begin()
    inspector = Inspector.from_engine(db.engine)

    # We need to re-create a minimal metadata with only the required things to
    # successfully emit drop constraints and tables commands for postgres (based
    # on the actual schema of the running instance)
    meta = MetaData()
    tables = []
    all_fkeys = []

    for table_name in inspector.get_table_names():
        fkeys = []

        for fkey in inspector.get_foreign_keys(table_name):
            if not fkey["name"]:
                continue

            fkeys.append(db.ForeignKeyConstraint((), (), name=fkey["name"]))

        tables.append(Table(table_name, meta, *fkeys))
        all_fkeys.extend(fkeys)

    for fkey in all_fkeys:
        con.execute(DropConstraint(fkey))

    for table in tables:
        con.execute(DropTable(table))

    trans.commit()

def import_articles(file):
    basename = os.path.basename(file)
    filename = basename.split('.')[0]
    a1 = Article(article=filename)
    db.session.add(a1)
    db.session.commit()

def get_file_by_type(filetype):
    sourcedir = app.config.get('TESTING_FOLDER')
    for basename in os.listdir(sourcedir):
        file = os.path.join(sourcedir, basename)
        basename = os.path.basename(file)
        extention = basename.split('.')[1]
        if extention == filetype:
            return file

def get_file_by_name(filename):
    sourcedir = app.config.get('TESTING_FOLDER')
    for basename in os.listdir(sourcedir):
        file = os.path.join(sourcedir, basename)
        basename = os.path.basename(file)
        if basename.startswith(filename):
            return file
    return file

In [4]:
import time
class Timer:
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, exc_type, value, tb):
        self.duration = time.time() - self.start

### Create table


In [5]:
delete_table_records()

In [7]:
db.drop_all()

In [8]:
db.create_all()

### import article data

In [10]:
import_articles(get_file_by_name('103_'))


In [11]:
# db.session.query(Article).delete()
# db.session.commit()


### tring

In [12]:
# db.session.remove()
# tokens = get_tokens(read_text(get_file('txt')))
# words = [Word(word=t) for t in tokens]
# db.session.add_all(words)
# db.session.commit()


In [13]:
# sentences = get_sentence(get_file('txt'))
# sentence = Sentence(sentence=sentences[0])
# tokens = get_tokens(sentences[0])
# words = [Word(word=t) for t in tokens]
# sw = [SentenceWord(sentence=sentence, word=w) for w in words]
#
# db.session.add_all(sw)
# db.session.new
# db.session.commit()

In [14]:
# db.session.remove()
# sentence = 'this is a one'
# tokens = get_tokens(sentence)
# a = db.session.query(Article).first()
# w1 = db.session.query(Word).filter(Word.word == tokens[0]).first()
# w2 = db.session.query(Word).filter(Word.word == tokens[1]).first()
# w3 = db.session.query(Word).filter(Word.word == tokens[2]).first()
# w4 = db.session.query(Word).filter(Word.word == tokens[3]).first()
# s = Sentence(sentence=sentence, article=a)
# sw1 = SentenceWord(word=w1)
# sw2 = SentenceWord(word=w2)
# sw3 = SentenceWord(word=w3)
# sw4 = SentenceWord(word=w4)
# s.sentencewords.append(sw1)
# s.sentencewords.append(sw2)
# s.sentencewords.append(sw3)
# s.sentencewords.append(sw4)
# db.session.add(s)
# db.session.commit()

### import words avoiding Duplicates

In [93]:
# insert article words
def import_word(filename):
    with Timer() as timer:
        tokens = set(get_tokens(read_text(get_file_by_name(filename))))
        exist = db.session.query(Word.word).all()
        exist =  set([e[0] for e in exist])
        not_exist = tokens - exist
        for n in not_exist:
            w = Word(word=n)
            db.session.add(w)
        db.session.new
        db.session.commit()
    print("took", timer.duration, "seconds")

db.session.remove()
import_word('big')

took 0.7895591259002686 seconds


### import sentence and sentenceword

In [104]:
def import_sentence(filename):
    with Timer() as timer:
        # insert SentenceWords
        db.session.remove()
        article = db.session.query(Article).first()

        sl = []
        sentences = get_sentence(get_file_by_name(filename))
        tokens_all = get_tokens(read_text(get_file_by_name(filename)))
        words_all = db.session.query(Word).filter(Word.word.in_(tokens_all)).all()
        for sentence in sentences:
            tokens = get_tokens(sentence)
            s = Sentence(sentence=sentence, article=article)
            w = [w for w in words_all if w.word in tokens]
            sw = [SentenceWord(word=i) for i in w]
            s.sentencewords = sw
            sl.append(s)
        db.session.add_all(sl)
        db.session.commit()

    print("took", timer.duration, "seconds")

db.session.remove()
import_sentence('103_')

took 2.2678542137145996 seconds


1885

### import my dict



In [47]:
def import_dict():
    sourcedir = app.config.get('MYDICT_FOLDER')
    file = os.path.join(sourcedir, 'mydict.csv' )
    with Timer() as timer:
        tokens = get_tokens(read_text(file))
        for t in tokens:
            m = db.session.query(Word).join(Mydict).filter(Word.word==t).first()
            if not m:
                w = db.session.query(Word).filter(Word.word == t).first()
                if not w:
                    w = Word(word=t)
                    m = Mydict(word=w)
                    db.session.add(w)
                    db.session.add(m)
                else:
                    m = Mydict(word=w)
                    db.session.add(m)
        db.session.new
        db.session.commit()
    print("took", timer.duration, "seconds")

import_dict()

took 5.542737245559692 seconds


## Querying Data

### all() method

In [18]:
db.session.query(Article).limit(10).all()

[<Article:1-103_The_One_With_the_Thumb>,
 <Article:2-103_The_One_With_the_Thumb>]

In [44]:
db.session.query(Word).get(10000)

<Word:10000-dedicated>

In [20]:
db.session.query(Sentence).limit(10).all()

[]

In [21]:
db.session.query(SentenceWord).limit(10).all()

[]

In [22]:
db.session.query(Mydict).limit(10).all()

[<Mydict:1-<Word:1-i>>, <Mydict:2-<Word:2-you>>, <Mydict:3-<Word:3-a>>]

### join() method


In [48]:
db.session.query(Word).join(Mydict).count()

4258

In [63]:
print(db.session.query(Word).filter(Word.word.in_(['Toby', 'Sarah'])))

SELECT words.id AS words_id, words.word AS words_word, words.translation AS words_translation 
FROM words 
WHERE words.word IN (%(word_1)s, %(word_2)s)


In [70]:
db.session.remove()
db.session.query(Word).filter(Word.word.in_(['the', 'one'])).all()



[<Word:6-the>, <Word:78-one>]

In [5]:
from sqlalchemy import distinct
db.session.query(
    Article, Word
).join(Sentence).join(SentenceWord).join(Word).filter(
    Article.id == 1,
).distinct().count()

772

In [10]:
wheres = db.session.query(
            Article, Word.word
        ).join(Sentence).join(SentenceWord).join(Word).filter(
            Article.id == 1,
        ).distinct().all()
words = [a[1] for a in wheres]
len(words)

772